In [16]:
import pandas as pd
import numpy as np
import json
import faiss
from datetime import datetime
import random
from sentence_transformers import SentenceTransformer , util
import requests

import mysql.connector.pooling

#MySQL DB 연결
with open('./config/db_config.json','r') as f:
    dbconfig = json.load(f)
    
cnxpool = mysql.connector.pooling.MySQLConnectionPool(pool_name = 'mypool',
                                                      pool_size = 3,
                                                      pool_reset_session=True,
                                                      **dbconfig) 

qestion_embedder = SentenceTransformer("./models/Huffon_sentence-klue-roberta-base")
index = faiss.read_index("./faiss/sts.index")

No sentence-transformers model found with name ./models/Huffon_sentence-klue-roberta-base. Creating a new one with MEAN pooling.


In [10]:
session_id = 1
ping_pong_url = f"https://builder.pingpong.us/api/builder/62c45862e4b0d7787e968b85/integration/v0.2/custom/{session_id}"
ping_pong_auth = "Basic a2V5OmVjNmEwNTQ1YTE2N2EzNWYwZmUzY2I0NGRiM2E0ZTk2"
headers = {"Authorization" : ping_pong_auth , "Content-Type" : "application/json"}

In [45]:
data = {"request": {"query": "안녕하세요"}}
data = {"request": {"query": "안녕하세요"}}
res = requests.post(ping_pong_url, headers = headers , json=data).json()

In [56]:
res['response']['replies'][0]['text']

'네 안녕안녕하세요!'

In [42]:
res

TypeError: 'Response' object is not subscriptable

In [41]:
res.text

'{"response":{"replies":[{"from":{"score":1.0,"name":"conversation","link":"/bot/62c45862e4b0d7787e968b85/conversation?scriptId=62c45862e4b0d7787e968bee","from":"대화 시나리오 / 안녕"},"type":"text","text":"안녕안녕입니다\\uD83D\\uDD90"}]},"version":"1.0.0"}'

In [2]:
def chatbot_answer(query : str ) -> str :
    
    #----------------------------#
    # Retrieval part             #
    #----------------------------#
    qestion_embedding = qestion_embedder.encode(query, normalize_embeddings=True,convert_to_tensor=True)
    distances, indices = index.search(np.expand_dims(qestion_embedding,axis=0),3)
    
    #max distance 가 0.5가 안넘으면 무슨말인지 모르겠어요 하고 NoSQL에 축적
    if np.min(distances[0]) > 0.6:
        data = {"request": {"query": query}}
        res = requests.post(ping_pong_url, headers = headers , json=data).json()
        #----NoSQL로 전송-------#
        #print(np.max(distances[0]))
        return res['response']['replies'][0]['text']
    
    index_q = [i[1] for i in zip(distances[0],indices[0]) if i[0] <= 0.6]
    index_q = list(map(lambda x : x+1, index_q))
    index_q = str(index_q).replace('[','(').replace(']',')')
    
    # MySQL DB에서 임베딩 벡터를 검색 Bi-Encoder
    cnx = cnxpool.get_connection()
    cursor = cnx.cursor(dictionary=True)
    
    sql = f"""SELECT `index` , user, system FROM chatbot WHERE `index` IN {index_q};"""
    cursor.execute(sql)
    res = cursor.fetchall()

    cnx.close()
    cursor.close()
    
    temp = pd.DataFrame(res)
    
    if len(temp) < 2 :
        return temp.loc[0].system
    
    return temp.loc[random.randint(0,1)].system

In [8]:
chatbot_answer("헤어졌어 여자친구랑")

'이제 그리워하지마세요.'